In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

Using TensorFlow backend.


In [2]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [3]:
def discriminator_model():
    model = Sequential()
    model.add(
            Conv2D(64, (5, 5),
            padding='same',
            input_shape=(28, 28, 1))
            )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [4]:
def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

In [5]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [20]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(12):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 200 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)

In [7]:
def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")

In [21]:
train(128)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch is 0
Number of batches 468
batch 0 d_loss : 0.762269
batch 0 g_loss : 0.733912
batch 1 d_loss : 0.742460
batch 1 g_loss : 0.717835
batch 2 d_loss : 0.726478
batch 2 g_loss : 0.707726
batch 3 d_loss : 0.708043
batch 3 g_loss : 0.698683
batch 4 d_loss : 0.695327
batch 4 g_loss : 0.683747
batch 5 d_loss : 0.670770
batch 5 g_loss : 0.673510
batch 6 d_loss : 0.650832
batch 6 g_loss : 0.658888
batch 7 d_loss : 0.629085
batch 7 g_loss : 0.638999
batch 8 d_loss : 0.602048
batch 8 g_loss : 0.633424
batch 9 d_loss : 0.593201
batch 9 g_loss : 0.617060
batch 10 d_loss : 0.577763
batch 10 g_loss : 0.604752
batch 11 d_loss : 0.557497
batch 11 g_loss : 0.589834
batch 12 d_loss : 0.538575
batch 12 g_loss : 0.580984
batch 13 d_loss : 0.524936
batch 13 g_loss : 0.570268
batch 14 d_loss : 0.515961
batch 14 g_loss : 0.557286
batch 15 d_loss : 0.512666
batch 15 g_loss : 0.555295
batch 16 d_loss : 0.497434
batch 16 g_loss : 0.543156
batch 17 d_loss : 0.490980
batch 17 g_loss : 0.529619
batch 18 d_loss

batch 150 d_loss : 0.305681
batch 150 g_loss : 0.773584
batch 151 d_loss : 0.303613
batch 151 g_loss : 0.796495
batch 152 d_loss : 0.306062
batch 152 g_loss : 0.811174
batch 153 d_loss : 0.301646
batch 153 g_loss : 0.848112
batch 154 d_loss : 0.308078
batch 154 g_loss : 0.850212
batch 155 d_loss : 0.320608
batch 155 g_loss : 0.894353
batch 156 d_loss : 0.313765
batch 156 g_loss : 0.910116
batch 157 d_loss : 0.295576
batch 157 g_loss : 0.920040
batch 158 d_loss : 0.294340
batch 158 g_loss : 0.983570
batch 159 d_loss : 0.275740
batch 159 g_loss : 0.971328
batch 160 d_loss : 0.267546
batch 160 g_loss : 1.018283
batch 161 d_loss : 0.255043
batch 161 g_loss : 1.097677
batch 162 d_loss : 0.260557
batch 162 g_loss : 1.082620
batch 163 d_loss : 0.258378
batch 163 g_loss : 1.129266
batch 164 d_loss : 0.273353
batch 164 g_loss : 1.126614
batch 165 d_loss : 0.256036
batch 165 g_loss : 1.202516
batch 166 d_loss : 0.265058
batch 166 g_loss : 1.237326
batch 167 d_loss : 0.232299
batch 167 g_loss : 1

batch 298 d_loss : 0.593882
batch 298 g_loss : 0.900915
batch 299 d_loss : 0.630623
batch 299 g_loss : 0.909600
batch 300 d_loss : 0.635897
batch 300 g_loss : 0.910164
batch 301 d_loss : 0.574472
batch 301 g_loss : 0.925157
batch 302 d_loss : 0.618814
batch 302 g_loss : 0.949014
batch 303 d_loss : 0.553881
batch 303 g_loss : 0.981560
batch 304 d_loss : 0.617518
batch 304 g_loss : 0.950228
batch 305 d_loss : 0.538325
batch 305 g_loss : 0.980844
batch 306 d_loss : 0.528669
batch 306 g_loss : 1.035999
batch 307 d_loss : 0.574031
batch 307 g_loss : 1.099507
batch 308 d_loss : 0.550887
batch 308 g_loss : 1.099791
batch 309 d_loss : 0.501566
batch 309 g_loss : 1.118963
batch 310 d_loss : 0.646195
batch 310 g_loss : 1.113103
batch 311 d_loss : 0.631709
batch 311 g_loss : 1.017301
batch 312 d_loss : 0.533872
batch 312 g_loss : 1.068798
batch 313 d_loss : 0.533255
batch 313 g_loss : 1.001054
batch 314 d_loss : 0.577705
batch 314 g_loss : 1.049531
batch 315 d_loss : 0.528230
batch 315 g_loss : 1

batch 446 d_loss : 0.481045
batch 446 g_loss : 0.993871
batch 447 d_loss : 0.487267
batch 447 g_loss : 0.992230
batch 448 d_loss : 0.533458
batch 448 g_loss : 1.038468
batch 449 d_loss : 0.490109
batch 449 g_loss : 1.018843
batch 450 d_loss : 0.497681
batch 450 g_loss : 1.078382
batch 451 d_loss : 0.495864
batch 451 g_loss : 1.008409
batch 452 d_loss : 0.549047
batch 452 g_loss : 1.034947
batch 453 d_loss : 0.498733
batch 453 g_loss : 1.026624
batch 454 d_loss : 0.492702
batch 454 g_loss : 1.011108
batch 455 d_loss : 0.500496
batch 455 g_loss : 1.065514
batch 456 d_loss : 0.513255
batch 456 g_loss : 1.059401
batch 457 d_loss : 0.481523
batch 457 g_loss : 1.056425
batch 458 d_loss : 0.444572
batch 458 g_loss : 1.062170
batch 459 d_loss : 0.464915
batch 459 g_loss : 1.087910
batch 460 d_loss : 0.466661
batch 460 g_loss : 1.072243
batch 461 d_loss : 0.430512
batch 461 g_loss : 1.103007
batch 462 d_loss : 0.497633
batch 462 g_loss : 1.101200
batch 463 d_loss : 0.498406
batch 463 g_loss : 1

...

batch 226 d_loss : 0.357649
batch 226 g_loss : 1.955165
batch 227 d_loss : 0.327654
batch 227 g_loss : 1.317449
batch 228 d_loss : 0.428912
batch 228 g_loss : 1.709842
batch 229 d_loss : 0.435534
batch 229 g_loss : 1.310177
batch 230 d_loss : 0.365036
batch 230 g_loss : 1.824129
batch 231 d_loss : 0.329981
batch 231 g_loss : 1.559273
batch 232 d_loss : 0.325549
batch 232 g_loss : 1.558028
batch 233 d_loss : 0.348165
batch 233 g_loss : 1.853416
batch 234 d_loss : 0.269246
batch 234 g_loss : 1.842019
batch 235 d_loss : 0.349632
batch 235 g_loss : 1.506458
batch 236 d_loss : 0.344444
batch 236 g_loss : 1.666634
batch 237 d_loss : 0.394353
batch 237 g_loss : 1.499349
batch 238 d_loss : 0.334581
batch 238 g_loss : 1.731288
batch 239 d_loss : 0.306947
batch 239 g_loss : 1.611405
batch 240 d_loss : 0.348549
batch 240 g_loss : 2.070685
batch 241 d_loss : 0.384255
batch 241 g_loss : 1.382543
batch 242 d_loss : 0.341413
batch 242 g_loss : 1.909578
batch 243 d_loss : 0.361337
batch 243 g_loss : 1

batch 374 d_loss : 0.405628
batch 374 g_loss : 2.000958
batch 375 d_loss : 0.473913
batch 375 g_loss : 1.485272
batch 376 d_loss : 0.364046
batch 376 g_loss : 2.589358
batch 377 d_loss : 0.289385
batch 377 g_loss : 2.109734
batch 378 d_loss : 0.308791
batch 378 g_loss : 2.184736
batch 379 d_loss : 0.331167
batch 379 g_loss : 2.256645
batch 380 d_loss : 0.456807
batch 380 g_loss : 2.587308
batch 381 d_loss : 0.582124
batch 381 g_loss : 1.905793
batch 382 d_loss : 0.684076
batch 382 g_loss : 0.856609
batch 383 d_loss : 0.760201
batch 383 g_loss : 1.303059
batch 384 d_loss : 0.551286
batch 384 g_loss : 1.547279
batch 385 d_loss : 0.438966
batch 385 g_loss : 1.279062
batch 386 d_loss : 0.347540
batch 386 g_loss : 2.066178
batch 387 d_loss : 0.267580
batch 387 g_loss : 2.033316
batch 388 d_loss : 0.248597
batch 388 g_loss : 2.500142
batch 389 d_loss : 0.261889
batch 389 g_loss : 3.018430
batch 390 d_loss : 0.290657
batch 390 g_loss : 3.258308
batch 391 d_loss : 0.303957
batch 391 g_loss : 2

batch 54 d_loss : 0.525907
batch 54 g_loss : 2.455119
batch 55 d_loss : 0.370698
batch 55 g_loss : 2.536176
batch 56 d_loss : 0.358193
batch 56 g_loss : 2.266137
batch 57 d_loss : 0.288632
batch 57 g_loss : 2.015559
batch 58 d_loss : 0.220300
batch 58 g_loss : 2.752158
batch 59 d_loss : 0.187626
batch 59 g_loss : 2.976782
batch 60 d_loss : 0.173600
batch 60 g_loss : 3.081774
batch 61 d_loss : 0.359394
batch 61 g_loss : 3.033190
batch 62 d_loss : 0.333062
batch 62 g_loss : 2.628735
batch 63 d_loss : 0.240698
batch 63 g_loss : 2.818357
batch 64 d_loss : 0.205516
batch 64 g_loss : 2.924166
batch 65 d_loss : 0.213193
batch 65 g_loss : 3.127401
batch 66 d_loss : 0.758226
batch 66 g_loss : 2.494548
batch 67 d_loss : 0.508743
batch 67 g_loss : 1.887325
batch 68 d_loss : 0.279597
batch 68 g_loss : 2.287123
batch 69 d_loss : 0.196402
batch 69 g_loss : 2.661697
batch 70 d_loss : 0.192431
batch 70 g_loss : 3.002649
batch 71 d_loss : 0.164131
batch 71 g_loss : 3.083231
batch 72 d_loss : 0.210384
b

batch 202 d_loss : 0.343325
batch 202 g_loss : 1.705688
batch 203 d_loss : 0.356358
batch 203 g_loss : 1.629241
batch 204 d_loss : 0.411172
batch 204 g_loss : 1.336097
batch 205 d_loss : 0.339632
batch 205 g_loss : 1.878001
batch 206 d_loss : 0.440848
batch 206 g_loss : 1.156678
batch 207 d_loss : 0.407016
batch 207 g_loss : 1.813476
batch 208 d_loss : 0.372185
batch 208 g_loss : 1.430210
batch 209 d_loss : 0.419311
batch 209 g_loss : 1.893216
batch 210 d_loss : 0.342931
batch 210 g_loss : 1.961701
batch 211 d_loss : 0.400810
batch 211 g_loss : 1.396561
batch 212 d_loss : 0.501640
batch 212 g_loss : 1.268517
batch 213 d_loss : 0.406693
batch 213 g_loss : 2.135548
batch 214 d_loss : 0.366798
batch 214 g_loss : 1.329088
batch 215 d_loss : 0.358144
batch 215 g_loss : 1.904796
batch 216 d_loss : 0.308122
batch 216 g_loss : 1.552570
batch 217 d_loss : 0.386617
batch 217 g_loss : 1.715416
batch 218 d_loss : 0.384160
batch 218 g_loss : 1.662020
batch 219 d_loss : 0.411942
batch 219 g_loss : 1

batch 350 d_loss : 0.421378
batch 350 g_loss : 1.146349
batch 351 d_loss : 0.496059
batch 351 g_loss : 2.148328
batch 352 d_loss : 0.385141
batch 352 g_loss : 1.376103
batch 353 d_loss : 0.324939
batch 353 g_loss : 2.086107
batch 354 d_loss : 0.320750
batch 354 g_loss : 2.164213
batch 355 d_loss : 0.322857
batch 355 g_loss : 1.952924
batch 356 d_loss : 0.351036
batch 356 g_loss : 1.732718
batch 357 d_loss : 0.421056
batch 357 g_loss : 1.428432
batch 358 d_loss : 0.406202
batch 358 g_loss : 2.226440
batch 359 d_loss : 0.488340
batch 359 g_loss : 0.768136
batch 360 d_loss : 0.462067
batch 360 g_loss : 2.216909
batch 361 d_loss : 0.350927
batch 361 g_loss : 1.917714
batch 362 d_loss : 0.365003
batch 362 g_loss : 1.519567
batch 363 d_loss : 0.333029
batch 363 g_loss : 1.438502
batch 364 d_loss : 0.352259
batch 364 g_loss : 1.555788
batch 365 d_loss : 0.328167
batch 365 g_loss : 2.103797
batch 366 d_loss : 0.372775
batch 366 g_loss : 1.553430
batch 367 d_loss : 0.365657
batch 367 g_loss : 1

In [ ]:
generate(132, True)

In [8]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

In [ ]:
if __name__ == "__main__":
    args = get_args()
    if args.mode == "train":
        train(BATCH_SIZE=args.batch_size)
    elif args.mode == "generate":
        generate(BATCH_SIZE=args.batch_size, nice=args.nice)